# 한국은행 Ecos api 사용하기

## ecos 홈페이지 : [한국은행경제통계시스템](https://ecos.bok.or.kr/)

- 주제별, 명칭별로 어떤 지표들이 있는지 확인해 볼 수 있다

## 통계 조회 조건 설정 : [한국은행 경제통계시스템 - OpenAPI 서비스](https://ecos.bok.or.kr/jsp/openapi/OpenApiController.jsp?t=guideServiceDtl&apiCode=OA-1040&menuGroup=MENU000004)

- 홈페이지에서 제공하는 지표들을 특정 조회조건과 코드를 이용해서 데이터를 다운받을 수 있는 사이트다. 개발자 가이드를 읽어보면 사용법이 자세히 나온다.

## 통계자료 (코드 및 단위) [한국은행 경제통계시스템 - OpenAPI 서비스](https://ecos.bok.or.kr/jsp/openapi/OpenApiController.jsp?t=guideStatCd&menuGroup=MENU000004&menuCode=MENU000024)

- 공공 사이트는 대부분 open api라는 것을 제공한다.

- 인터넷 이용자가 일방적으로 웹 검색 결과 및 사용자인터페이스(UI) 등을 제공받는 데 그치지 않고 직접 응용 프로그램과 서비스를 개발할 수 있도록 공개된 API를 말한다. 

- 지도 서비스 및 다양한 서비스에서 시도되고 있으며 누구나 접근하여 사용할 수 있다는 장점이 있다.

- 그 중 우리는 한국은행에서 제공하는 ecos api를 사용할 것이다.

- 비슷하게 재무 데이터를 이용하려면 dart의 open api를 사용하면 된다.

In [ ]:
# 필요한 패키지들을 임포트한다.
# 웹 요청하는 모듈은 import requests 도 있지만 urllib도 있다. 기본적으로 파이썬에서 제공하는 표준라이브러리이다.
# request보다 사용법이 어렵고 더 고수준 HTTP 클라이언트 인터페이스로 Requests 패키지를 권장하지만 한번 사용해본다.

from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import json
import os
 

# 한국은행API 인증코드. 본인 아이디의 인증코드를 입력한다.
# 개인 인증키라 공개 깃허브에 올릴 수 없어 약간의 처리를 해주었다.
secret_file = os.path.join(BASE_DIR, 'secrets.json') # secrets.json 파일 위치를 명시

with open(secret_file) as f:
    secrets = json.loads(f.read())

def get_secret(setting, secrets=secrets):
    """비밀 변수를 가져오거나 명시적 예외를 반환한다."""
    try:
        return secrets[setting]
    except KeyError:
        print(f"Set the {setting} environment variable")


# key에 회원가입 및 open api에서 신청한 secret key를 입력한다.
# 이 자료를 읽는 다른 분들은 위에 모듈 import 한거빼고 다 주석처리하고 key="개인비밀키" 하면된다.
key = get_secret("SECRET_KEY")
print(key)

In [ ]:

key = '5AO6I682B0L73A3FH2DB'
 
# https://ecos.bok.or.kr/jsp/openapi/OpenApiController.jsp?t=guideServiceDtl&apiCode=OA-1040&menuGroup=MENU000004
# 여기서 주소 만들면 쉽다.
url="http://ecos.bok.or.kr/api/StatisticSearch/"+key+"/json/kr/1/10000/060Y001/DD/190001/202008/010200000/?/?/"

result = urlopen(url)
html = result.read()
data = json.loads(html)
# print(data)
data=data["StatisticSearch"]["row"]
Interest_rate = pd.DataFrame(data)
Interest_rate = Interest_rate[Interest_rate["ITEM_CODE1"]=="010200000"]
Interest_rate["DATA_VALUE"]=Interest_rate["DATA_VALUE"].astype("float")
# print(Interest_rate)
 
Interest_rate["TIME"] = Interest_rate["TIME"].astype("str")
Interest_rate["TIME"] = Interest_rate["TIME"].apply(lambda x: datetime.datetime.strptime(x, '%Y%m%d'))
# print(Interest_rate["TIME"])

Interest_rate=Interest_rate.sort_values("TIME")
Interest_rate.set_index("TIME",inplace=True)
# print(Interest_rate)

Interest_rate["DATA_VALUE"].plot()


In [5]:
# xlabel=Interest_rate["TIME"]
# xlabel = xlabel.drop_duplicates()
# plt.scatter(Interest_rate.index,Interest_rate["DATA_VALUE"])
# plt.plot(Interest_rate.index,Interest_rate["DATA_VALUE"])
# plt.xticks(Interest_rate.index, xlabel,rotation=70)
# plt.show()

# Plotly 시각화 패키지

- 사이트 https://plotly.com/python/

![plotly](./plotly.png)

- matplotlib, seaborn 보다 훨씬 세련된 동적인 그래프를 그릴 수 있다.

- d3.js를 이용하여 보다 interactive하게 그래프를 만들어준다. (D3.js는 웹브라우저 상에서 동적이고 인터렉티브한 정보시각화를 구현하기 위한 자바스크립트 라이브러리)

- 동적으로 확대, 확장, 이동, 회전 가능하며, 마우스를 대면 값 표시도 된다.

- 응용은 여기 참고 : https://www.kaggle.com/hakkisimsek/plotly-tutorial-3



In [6]:
import plotly.graph_objs as go

# 오프라인 모드를 사용한다. 온라인모드는 유료다.
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# 주피터 노트북이나 파이썬 개발환경 내에서 그래프를 보여주는 것을 선택한다.
init_notebook_mode(connected=True)

# Plotting : 2차원 데이터 분포를 그리기 위해 go.Scatter()를 사용한다.
# matplotlib과 마찬가지로 x,y에는 iterable한 자료가 들어가면 된다.
# 이것을 최종적으로 trace라는 변수에 할당해준다.
trace = go.Scatter(x=Interest_rate.index, y=Interest_rate["DATA_VALUE"]) 

# go.Layout()을 사용해 그래프의 세부 디테일을 지정해줄 수 있다. matplotlib의 figure와 axes, axis를 변경하는 것과 같다.
layout = go.Layout( title='국고채(3년) 금리 추이', 
                    xaxis=dict(

                            # range selector 표시하기 
                            rangeselector=dict( 
                                buttons=list(
                                    [ dict(count=1, label='1m', step='month', stepmode='backward'), 
                                        dict(count=3, label='3m', step='month', stepmode='backward'), 
                                        dict(count=6, label='6m', step='month', stepmode='backward'), 
                                        dict(count=1, label="YTD", step="year", stepmode="todate"),
                                        dict(count=1, label="1y", step="year", stepmode="backward"),
                                        dict(count=2, label="2y", step="year", stepmode="backward"),
                                        dict(count=3, label="3y", step="year", stepmode="backward"),
                                        dict(count=5, label="5y", step="year", stepmode="backward"),
                                        dict(count=7, label="7y", step="year", stepmode="backward"),
                                        dict(count=10, label="10y", step="year", stepmode="backward"),
                                        dict(step='all') ]) 
                                    ), 
                            
                            # range slider 표시하기
                            rangeslider=dict(visible=True), 
                            type='date',
                            title='날짜'
                            ),
                        yaxis=dict(title='수익률'),
                        width=800, height=600
                        )

# range selector과 range slider에 대해 다양한 옵션은 여기 참고 : https://plotly.com/python/range-slider/

print(trace)
print(layout)

In [ ]:
data1=[trace] 
fig1 = dict(data=data1, layout=layout)
iplot(fig1)